In [255]:
import pandas as pd
from annotate_papers import *

# read metadata df

In [230]:
metadata_df = pd.read_excel("12797/dataframe12797.xlsx")
metadata_df.head()

,Unnamed: 0,author,title,affiliation,email,abstract,year,journal,doi,street,city,zip,filenames
0,0,"Michael L. Brodie, Michael Stonebraker",DARWIN: On the Incremental Migration of Legacy...,Hochschule Albstadt-Sigmaringen,michael_l._brodie@hochschule_albstadt_sigmarin...,befragungen von mitarbeitern der verwaltung ha...,1997,ZA-Information / Zentralarchiv für Empirische ...,https://doi.org/10.1080/00036840701736115,Schoenebergerstrasse 79,Stralsund,18412,12797_0.docx
1,1,David Beech,Unification of Value and Object ADTs,Hochschule für evangelische Kirchenmusik Bayreuth,david_beech@hochschule_für_evangelische_kirche...,"Wer Daten mittels Umfragen erhebt, kennt das P...",1996,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Sonnenallee 88,Augsburg,86015,12797_1.docx
2,2,Brigitte Bartsch-Spärl,Zur Systemarchitektur von LILOG,bbw Hochschule,brigitte_bartsch_spärl@bbw_hochschule.de,'im mikrozensus werden unterschiedliche konzep...,2000,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Neue Roßstr. 2,Esslingen Hegensberg,73732,12797_2.docx
3,3,Egon Bärger,A Logical Operational Semantics of Full Prolog,Evangelische Hochschule Berlin,egon_bärger@evangelische_hochschule_berlin.de,'in diesem bericht werden die wichtigsten maße...,2000,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Paderborner Strasse 56,Bobingen,86399,12797_3.docx
4,4,Roland Seiffert,How could a good system of practical NLP look ...,"Hochschule für Medien, Kommunikation und Wirts...","roland_seiffert@hochschule_für_medien,_kommuni...",'im zentralarchiv stehen mehr als 200 studien ...,1996,ZA-Information / Zentralarchiv für Empirische ...,https://doi.org/10.1080/00036840701736115,Stuttgarter Platz 43,Olfen,59399,12797_4.docx


In [231]:
metadata_df.rename(columns={'year':'date', 'city': 'address'}, inplace=True)

In [232]:
metadata_df[metadata_df.filenames == "12797_1063.docx"]

,Unnamed: 0,author,title,affiliation,email,abstract,date,journal,doi,street,address,zip,filenames
1063,1063,"Franco Blanchini, Giulia Giordano",BDC-Decomposition for Global Influence Analysis.,Georg-August-Universität Göttingen,franco_blanchini@georg_august_universität_gött...,Am 20. Februar 2001 übergab der russische Vert...,2001,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Paul-Nevermann-Platz 70,Bad Neustadt,97606,12797_1063.docx


# Subclass for 12797

In [240]:
class Template_12797(Paper):
    def __init__(self, filename, metadata_dict):
        super().__init__(filename, metadata_dict)
        
    def get_metadata_items(self, metadata_df):
        reg = re.compile("[^/]+$")
        instance = re.search(reg, self.filename)[0][:-4] + ".docx"
        
        current = metadata_df[metadata_df["filenames"] == instance]
        for key in self.metadata.keys():
            if key != "journal":
                self.metadata[key] += str(list(current[key])[0])
    
    def bbox_from_text(self):
        # get the first page
        doc = fitz.open(self.filename)
        page = doc[0]
        
        if list(self.metadata.values())[0] == "":
            raise MetadataError("No metadata values specified. Run method get_metadata_items(metadata_df) before callsing this method.", "")
       
        textWords = page.getTextWords()
        ending_box = None
        for key, val in self.metadata.items():
            if key == "title" and len(val) <= 50:
                title_rects = []
                for word in textWords:
                    if str.lower(word[4]) in str.lower(val) and word[6] == 0:
                        title_rects.append(fitz.Rect(word[:4]))
                self.rectangles[key] = title_rects
            elif key == "date":
                date_rect = []
                for word in textWords:
                    if str.lower(word[4]) in str.lower(val) and word[6] == 1:
                        date_rect.append(fitz.Rect(word[:4]))
                self.rectangles[key] = date_rect
            elif key == "address":
                ending_box = page.searchFor("Zusammenfassung")   
                self.rectangles[key] = [rect for rect in page.searchFor(val) if rect[3] < ending_box[0][3]]
            else:
                self.rectangles[key] = page.searchFor(val, hit_max=1000)
                if len(self.rectangles[key]) == 0:
                    text = self.metadata[key].split("-")
                    for part in text:
                        results = page.searchFor(part, hit_max=1000)
                        self.rectangles[key] += results
                        
            if len(self.rectangles[key]) > 1:
                foundRectList = [tuple(rect) for rect in self.rectangles[key]]
    
                #find min x, min y, max x, max y
                min_x = foundRectList[0][0]
                min_y = foundRectList[0][1]
                max_x = foundRectList[0][2]
                max_y = foundRectList[0][3]

                for rect in foundRectList:
                    if rect[0] < min_x:
                        min_x = rect[0]
                    if rect[1] < min_y:
                        min_y = rect[1]
                    if rect[2] > max_x:
                        max_x = rect[2]
                    if rect[3] > max_y:
                        max_y = rect[3]
            
                self.rectangles[key] = [fitz.Rect(min_x, min_y, max_x, max_y)]
                
            if ending_box != None and len(ending_box) > 1:
                ending_box = [rect for rect in ending_box if rect[3] > self.rectangles["title"][3] and rect[3] < self.rectangles["abstract"][3]]
                self.rectangles["address"] = [rect for rect in page.searchFor(val) if rect[3] < ending_box[0][3]]
                if len(self.rectangles["address"]) > 1:
                    foundRectList = [tuple(rect) for rect in self.rectangles["address"]]

                    #find min x, min y, max x, max y
                    min_x = foundRectList[0][0]
                    min_y = foundRectList[0][1]
                    max_x = foundRectList[0][2]
                    max_y = foundRectList[0][3]

                    for rect in foundRectList:
                        if rect[0] < min_x:
                            min_x = rect[0]
                        if rect[1] < min_y:
                            min_y = rect[1]
                        if rect[2] > max_x:
                            max_x = rect[2]
                        if rect[3] > max_y:
                            max_y = rect[3]

                    self.rectangles["address"] = [fitz.Rect(min_x, min_y, max_x, max_y)]


# Annotate all pdfs of template 12797

In [241]:
from glob import glob

In [242]:
pdfs = glob("12797/*.pdf")
pdfs

['12797/12797_1149.pdf',
 '12797/12797_1699.pdf',
 '12797/12797_1641.pdf',
 '12797/12797_1139.pdf',
 '12797/12797_294.pdf',
 '12797/12797_712.pdf',
 '12797/12797_371.pdf',
 '12797/12797_263.pdf',
 '12797/12797_109.pdf',
 '12797/12797_501.pdf',
 '12797/12797_536.pdf',
 '12797/12797_1562.pdf',
 '12797/12797_415.pdf',
 '12797/12797_7.pdf',
 '12797/12797_497.pdf',
 '12797/12797_1100.pdf',
 '12797/12797_1506.pdf',
 '12797/12797_837.pdf',
 '12797/12797_34.pdf',
 '12797/12797_746.pdf',
 '12797/12797_1698.pdf',
 '12797/12797_826.pdf',
 '12797/12797_134.pdf',
 '12797/12797_794.pdf',
 '12797/12797_1284.pdf',
 '12797/12797_954.pdf',
 '12797/12797_812.pdf',
 '12797/12797_783.pdf',
 '12797/12797_278.pdf',
 '12797/12797_89.pdf',
 '12797/12797_260.pdf',
 '12797/12797_1577.pdf',
 '12797/12797_1282.pdf',
 '12797/12797_1287.pdf',
 '12797/12797_1436.pdf',
 '12797/12797_1192.pdf',
 '12797/12797_203.pdf',
 '12797/12797_346.pdf',
 '12797/12797_841.pdf',
 '12797/12797_1032.pdf',
 '12797/12797_132.pdf',
 '127

In [248]:
errors = []
for pdf in pdfs:
    
    print(pdf)
    
    p = Template_12797(pdf, {"title": "", "author":"", "abstract": "", "journal": "kommunikation@gesellschaft", "date": "", "address": ""})
    try:
        p.resize_image(569,794)
        p.save_image("12797_annotations/")
        p.get_metadata_items(metadata_df)
        p.bbox_from_text()
        #print(p.rectangles)
        p.get_annotations()
        p.save_annotations("12797_annotations/")
    except:
        errors.append(pdf)
        pass


12797/12797_1149.pdf
12797/12797_1699.pdf
12797/12797_1641.pdf
12797/12797_1139.pdf
12797/12797_294.pdf
12797/12797_712.pdf
12797/12797_371.pdf
12797/12797_263.pdf
12797/12797_109.pdf
12797/12797_501.pdf
12797/12797_536.pdf
12797/12797_1562.pdf
12797/12797_415.pdf
12797/12797_7.pdf
12797/12797_497.pdf
12797/12797_1100.pdf
12797/12797_1506.pdf
12797/12797_837.pdf
12797/12797_34.pdf
12797/12797_746.pdf
12797/12797_1698.pdf
12797/12797_826.pdf
12797/12797_134.pdf
12797/12797_794.pdf
12797/12797_1284.pdf
12797/12797_954.pdf
12797/12797_812.pdf
12797/12797_783.pdf
12797/12797_278.pdf
12797/12797_89.pdf
12797/12797_260.pdf
12797/12797_1577.pdf
12797/12797_1282.pdf
12797/12797_1287.pdf
12797/12797_1436.pdf
12797/12797_1192.pdf
12797/12797_203.pdf
12797/12797_346.pdf
12797/12797_841.pdf
12797/12797_1032.pdf
12797/12797_132.pdf
12797/12797_1549.pdf
12797/12797_705.pdf
12797/12797_1079.pdf
12797/12797_250.pdf
12797/12797_1129.pdf
12797/12797_942.pdf
12797/12797_550.pdf
12797/12797_1414.pdf
12797

12797/12797_37.pdf
12797/12797_1319.pdf
12797/12797_1455.pdf
12797/12797_589.pdf
12797/12797_751.pdf
12797/12797_1342.pdf
12797/12797_187.pdf
12797/12797_1069.pdf
12797/12797_1175.pdf
12797/12797_112.pdf
12797/12797_1559.pdf
12797/12797_449.pdf
12797/12797_383.pdf
12797/12797_1460.pdf
12797/12797_955.pdf
12797/12797_69.pdf
12797/12797_102.pdf
12797/12797_1236.pdf
12797/12797_1615.pdf
12797/12797_1002.pdf
12797/12797_377.pdf
12797/12797_221.pdf
12797/12797_1594.pdf
12797/12797_1337.pdf
12797/12797_923.pdf
12797/12797_906.pdf
12797/12797_322.pdf
12797/12797_1692.pdf
12797/12797_1209.pdf
12797/12797_394.pdf
12797/12797_1320.pdf
12797/12797_1046.pdf
12797/12797_554.pdf
12797/12797_1606.pdf
12797/12797_1691.pdf
12797/12797_1561.pdf
12797/12797_860.pdf
12797/12797_214.pdf
12797/12797_532.pdf
12797/12797_539.pdf
12797/12797_847.pdf
12797/12797_862.pdf
12797/12797_1448.pdf
12797/12797_821.pdf
12797/12797_375.pdf
12797/12797_1023.pdf
12797/12797_54.pdf
12797/12797_978.pdf
12797/12797_1586.pdf
1

12797/12797_873.pdf
12797/12797_473.pdf
12797/12797_36.pdf
12797/12797_1276.pdf
12797/12797_1151.pdf
12797/12797_566.pdf
12797/12797_1485.pdf
12797/12797_1137.pdf
12797/12797_172.pdf
12797/12797_1465.pdf
12797/12797_14.pdf
12797/12797_1605.pdf
12797/12797_1194.pdf
12797/12797_889.pdf
12797/12797_1513.pdf
12797/12797_535.pdf
12797/12797_40.pdf
12797/12797_376.pdf
12797/12797_438.pdf
12797/12797_1600.pdf
12797/12797_750.pdf
12797/12797_1022.pdf
12797/12797_913.pdf
12797/12797_373.pdf
12797/12797_1160.pdf
12797/12797_1649.pdf
12797/12797_229.pdf
12797/12797_1080.pdf
12797/12797_1572.pdf
12797/12797_793.pdf
12797/12797_411.pdf
12797/12797_159.pdf
12797/12797_82.pdf
12797/12797_1543.pdf
12797/12797_1251.pdf
12797/12797_702.pdf
12797/12797_1344.pdf
12797/12797_1452.pdf
12797/12797_1544.pdf
12797/12797_1028.pdf
12797/12797_572.pdf
12797/12797_661.pdf
12797/12797_333.pdf
12797/12797_897.pdf
12797/12797_664.pdf
12797/12797_682.pdf
12797/12797_21.pdf
12797/12797_609.pdf
12797/12797_140.pdf
12797

12797/12797_1297.pdf
12797/12797_988.pdf
12797/12797_1569.pdf
12797/12797_195.pdf
12797/12797_283.pdf
12797/12797_1176.pdf
12797/12797_822.pdf
12797/12797_909.pdf
12797/12797_867.pdf
12797/12797_1468.pdf
12797/12797_1238.pdf
12797/12797_184.pdf
12797/12797_51.pdf
12797/12797_1308.pdf
12797/12797_513.pdf
12797/12797_529.pdf
12797/12797_1629.pdf
12797/12797_902.pdf
12797/12797_1618.pdf
12797/12797_55.pdf
12797/12797_262.pdf
12797/12797_420.pdf
12797/12797_1150.pdf
12797/12797_1260.pdf
12797/12797_599.pdf
12797/12797_657.pdf
12797/12797_1655.pdf
12797/12797_825.pdf
12797/12797_905.pdf
12797/12797_1469.pdf
12797/12797_600.pdf
12797/12797_1128.pdf
12797/12797_1396.pdf
12797/12797_1473.pdf
12797/12797_482.pdf
12797/12797_805.pdf
12797/12797_1579.pdf
12797/12797_830.pdf
12797/12797_218.pdf
12797/12797_108.pdf
12797/12797_1668.pdf
12797/12797_311.pdf
12797/12797_299.pdf
12797/12797_1550.pdf
12797/12797_755.pdf
12797/12797_547.pdf
12797/12797_1170.pdf
12797/12797_1146.pdf
12797/12797_582.pdf
12

12797/12797_524.pdf
12797/12797_1178.pdf
12797/12797_640.pdf
12797/12797_1328.pdf
12797/12797_43.pdf
12797/12797_503.pdf
12797/12797_1258.pdf
12797/12797_350.pdf
12797/12797_1130.pdf
12797/12797_226.pdf
12797/12797_507.pdf
12797/12797_639.pdf
12797/12797_1526.pdf
12797/12797_934.pdf
12797/12797_930.pdf
12797/12797_762.pdf
12797/12797_1625.pdf
12797/12797_587.pdf
12797/12797_1154.pdf
12797/12797_1001.pdf
12797/12797_1451.pdf
12797/12797_489.pdf
12797/12797_1136.pdf
12797/12797_450.pdf
12797/12797_1193.pdf
12797/12797_164.pdf
12797/12797_1003.pdf
12797/12797_255.pdf
12797/12797_170.pdf
12797/12797_176.pdf
12797/12797_1206.pdf
12797/12797_1636.pdf
12797/12797_1095.pdf
12797/12797_441.pdf
12797/12797_494.pdf
12797/12797_786.pdf
12797/12797_422.pdf
12797/12797_1169.pdf
12797/12797_565.pdf
12797/12797_1256.pdf
12797/12797_1584.pdf
12797/12797_1696.pdf
12797/12797_1665.pdf
12797/12797_844.pdf
12797/12797_1101.pdf
12797/12797_834.pdf
12797/12797_1411.pdf
12797/12797_1312.pdf
12797/12797_1042.p

In [249]:
errors

['12797/12797_800.pdf',
 '12797/12797_1102.pdf',
 '12797/12797_1585.pdf',
 '12797/12797_1188.pdf',
 '12797/12797_1376.pdf',
 '12797/12797_622.pdf',
 '12797/12797_1396.pdf',
 '12797/12797_171.pdf',
 '12797/12797_930.pdf']

In [254]:
import os
for pdf in errors:
    print(pdf[6:-4] + ".jpeg")
    jpeg = "12797_annotations/" + pdf[6:-4] + ".jpeg"
    json = "12797_annotations/" + pdf[6:-4] + ".json"
    if os.path.exists(jpeg):
        os.remove(jpeg)
        print("removed jpeg: " + jpeg)
    if os.path.exists(json):
        os.remove(json)
        print("removed json: " + json)
    

12797_800.jpeg
12797_1102.jpeg
removed jpeg: 12797_annotations/12797_1102.jpeg
12797_1585.jpeg
removed jpeg: 12797_annotations/12797_1585.jpeg
12797_1188.jpeg
removed jpeg: 12797_annotations/12797_1188.jpeg
12797_1376.jpeg
removed jpeg: 12797_annotations/12797_1376.jpeg
12797_622.jpeg
removed jpeg: 12797_annotations/12797_622.jpeg
12797_1396.jpeg
removed jpeg: 12797_annotations/12797_1396.jpeg
12797_171.jpeg
removed jpeg: 12797_annotations/12797_171.jpeg
12797_930.jpeg
removed jpeg: 12797_annotations/12797_930.jpeg


In [246]:
doc = fitz.open("12797/12797_800.pdf")
page = doc[0]

In [506]:
page.searchFor('ökonomischen Panels (SOEP) zurückgegriffen, um die zeitliche Entwicklung des subjektiven Wohlbefindens in West')

[Rect(71.06400299072266, 473.1219787597656, 523.5271606445312, 488.29095458984375),
 Rect(71.06400299072266, 488.84197998046875, 183.6830596923828, 504.0109558105469)]